In [ ]:
!pip install torch torchvision #installing pytorch framework

In [ ]:
!pip install pillow

In [ ]:
train_csv_path="/kaggle/input/converting-handwritten-equations-to-latex-code/col_774_A4_2023/HandwrittenData/train_hw.csv"
#train_csv_path='/kaggle/input/converting-handwritten-equations-to-latex-code/col_774_A4_2023/SyntheticData/train.csv'
image_path='/kaggle/input/converting-handwritten-equations-to-latex-code/col_774_A4_2023/SyntheticData/images'

In [ ]:
import os
import cv2
import csv
from PIL import Image
import IPython.display as display
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch


In [ ]:
csv_path = train_csv_path
i=0
with open(csv_path, 'r') as file:
    csv_reader = csv.reader(file)
     
    for row in csv_reader:
        if(i>=0):
            print(row)    
        i=i+1
        if(i>=4):
            break



In [ ]:
input_string = "This is a sample string for tokenization"
# Split the string into tokens
tokens = input_string.split(' ')
# Print the tokens
print(tokens)

**vocab**

In [ ]:
formula_to_tokens={}#formulat number to token dictionary
tokens_set=set()#all unique tokens
#spliting making formula into token(.spllit(" ")): making all theeir length same by adding <PAD> token , also adding
#<SOS> start and end <EOS> token also
csv_path = train_csv_path
i=0
max_length=0
with open(csv_path, 'r') as file:
    csv_reader = csv.reader(file)
     
    for row in csv_reader:
        if(i>0):
            tokens=row[1].split(' ')
            tokens_set |= set(tokens)
            
            #formula_to_token[i]=tokens
            max_length= max(max_length, len(tokens))
            #print(i,len(tokens))
        i=i+1

tokens_set.add("<PAD>")
tokens_set.add("<SOS>")
tokens_set.add("<EOS>")
tokens_idx = {value: idx for idx, value in enumerate(tokens_set)}
print(tokens_idx)

In [ ]:
print(tokens_idx["<EOS>"])
print(tokens_idx["<SOS>"])

In [ ]:
i=0
with open(csv_path, 'r') as file:
    csv_reader = csv.reader(file)
     
    for row in csv_reader:
        #print(i)
        if(i>0):
            tokens=row[1].split(' ')
            tokens = [tokens_idx[token] for token in tokens]
            while(len(tokens)<max_length):
                tokens.append(tokens_idx["<PAD>"])
            tokens.insert(0,tokens_idx["<SOS>"] )
            tokens.append(tokens_idx["<EOS>"])
            formula_to_tokens[i-1]=tokens
            #max_length= max(max_length, len(tokens))
            #print("hi")
            #print(i,tokens)
        i=i+1


In [ ]:
def formula_to_token(formula):
    tokens=formula.split(' ')
    tokens = [tokens_idx[token] for token in tokens]
    while(len(tokens)<max_length):
            tokens.append(tokens_idx["<PAD>"])
    tokens.insert(0,tokens_idx["<SOS>"] )
    tokens.append(tokens_idx["<EOS>"])
    return tokens
    

In [ ]:
print(formula_to_token("$") )

In [ ]:
# print(max_length)
# print(formula_to_tokens)
# print(" ")
# print(tokens_set)

In [ ]:

from PIL import Image
import torchvision.transforms as transforms

def preprocess(image_path):
    # Read the image using PIL
    img = Image.open(image_path).convert("L")  # Convert to grayscale
    
    # Resize the image to 224x224
    img_resized = img.resize((224, 224))

    # Create R, G, B channels by duplicating the grayscale image
    r_channel = img_resized.copy()
    g_channel = img_resized.copy()
    b_channel = img_resized.copy()

    # Merge the three channels to create an RGB image
    rgb_image = Image.merge("RGB", (r_channel, g_channel, b_channel))

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    rgb_image = transform(rgb_image)
    
    return rgb_image


In [ ]:
image_path="/kaggle/input/converting-handwritten-equations-to-latex-code/col_774_A4_2023/HandwrittenData/images/test/001-equation000.png"
# Open the image file

img= preprocess(image_path)
print(img.shape)

**Preprocess and Loading of data**

In [ ]:
import pandas as pd
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Assuming your CSV file looks like this:
# | image_name | formula       |
# |------------|---------------|
# | image1.png | \alpha + \beta|
# | image2.png | \gamma \times \delta|
# ...

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.data)
    def preprocess(image_path):
        # Read the image using PIL
        img = Image.open(image_path).convert("L")  # Convert to grayscale

        # Resize the image to 224x224
        img_resized = img.resize((224, 224))

        # Create R, G, B channels by duplicating the grayscale image
        r_channel = img_resized.copy()
        g_channel = img_resized.copy()
        b_channel = img_resized.copy()

        # Merge the three channels to create an RGB image
        rgb_image = Image.merge("RGB", (r_channel, g_channel, b_channel))

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        rgb_image = transform(rgb_image)

        return rgb_image
        
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        img_name = self.data["image"][idx]
        img_path=root_dir+"/"+img_name
        image=preprocess(img_path)
        formula = torch.tensor(formula_to_tokens[idx])
        return image, formula

    

root_dir='/kaggle/input/converting-handwritten-equations-to-latex-code/col_774_A4_2023/HandwrittenData/images/train'
batch_size=32
train_dataset = CustomDataset(train_csv_path, root_dir)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
for row in train_dataset:
    print(row[0].shape,row[1].shape)
    break

In [ ]:

for i, batch in enumerate(train_loader):
    img, formula = batch
    print(img.shape,formula.shape)
    break

**Encoder**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        # Define the layers for the CNN
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=5)
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=5)
        self.pool5 = nn.MaxPool2d(kernel_size=2)
        self.avgpool = nn.AvgPool2d(kernel_size=3, stride=1)  # Output size: 1x1x512

    def forward(self, x):
        # Forward pass through the CNN layers
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = self.pool5(F.relu(self.conv5(x)))
        x = self.avgpool(x)
        x = x.squeeze()
        x = x.unsqueeze(dim = 1)
        return x




In [ ]:
encoder = Encoder()

for i, batch in enumerate(train_loader):
    img, formula = batch
    print(encoder(img).shape)
    break
    

**Decoder**

In [ ]:
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Decoder, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim + 512, hidden_dim, batch_first=True)
        
        # Output layer
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden, cell,context_vec):
        # x: input index-token
        # hidden: tuple (hidden state, cell state)
        
        # Embedding
        
        embedded = self.embedding(x) #1x512
        lstm_input=torch.cat((context_vec,embedded), dim = 2)#1x1024
        
        # LSTM
        output,(hidden,cell) = self.lstm(lstm_input, (hidden,cell) )
        
        
#         # Output layer
        output = self.fc(output)
        
        return output, hidden,cell

# Example instantiation
vocab_size = len(tokens_set)  # Replace with the actual size of your vocabulary
embedding_dim = 512
hidden_dim = 512

# Instantiate the decoder
decoder = Decoder(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim)


**FINAL CODE**

In [ ]:
for i, batch in enumerate(train_loader):
    img, formula = batch
    print(img.shape,formula.shape)
    break

In [ ]:
 class combined(nn.Module):
        def __init__(self, encoder, decoder):
            super(combined, self).__init__()
            self.encoder = encoder
            self.decoder = decoder
        
        def forward(self, images, formula):
            print("image shape ",images.shape)
            context_vec = encoder(images)#32x1x512  -- formuala- #32x88
            print(" context_vec shape ",context_vec.shape)
            hidden=torch.zeros(1, context_vec.shape[0], context_vec.shape[2] )
            cell=torch.zeros(1, context_vec.shape[0], context_vec.shape[2] )
            #print(context_vec.shape)
            input=formulas[:,0]
            input=input.unsqueeze(1)#32x1
            target_len = formulas.shape[1]
            batch_size= formulas.shape[0]

            result = torch.zeros(target_len, batch_size, vocab_size)
            input = formulas[:, 0]
            for t in range(0 , target_len):
                input=input.unsqueeze(1)#32x1
                #print(input.shape)
                output, hidden,cell=decoder(input,hidden,cell,context_vec)
                output = output.reshape((output.shape[0], output.shape[2]))
                result[t] = output
                teacher = random.random() < 0.5  
                
                if teacher == 0:
                    input = output.argmax(dim = 1)
                    #print(t," ",input.shape)
                else:
                    input = formulas[:, t]
            return result
        
        def predict(self,images):
            context_vec = encoder(images)#32x1x512  -- formuala- #32x88
            hidden=torch.zeros(1, context_vec.shape[0], context_vec.shape[2] )
            cell=torch.zeros(1, context_vec.shape[0], context_vec.shape[2] )
            #print(context_vec.shape)
        
            target_len = 9999999
            batch_size= context_vec.shape[0]
            

            result =[]
            input = torch.full((batch_size,), 101)
            for t in range(0 , target_len):
                input=input.unsqueeze(1)#32x1
                #print(input.shape)
                output, hidden,cell=decoder(input,hidden,cell,context_vec)
                output = output.reshape((output.shape[0], output.shape[2]))
                myout=output.argmax(dim = 1)
                print(t,myout[0])
                result.append(myout[0])
                if(myout[0] == 7):#112
                    return result
                input=myout
            
            result=torch.tensor(result)
            result= torch.stack(result)
            return result
            
            
        
            

In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# Assuming you have a SimpleLSTM model and other necessary components
# (model, criterion, train_loader, etc.)

# Define your optimizer

Final_model = combined(Encoder(), Decoder(vocab_size, 512, 512))
optimizer = optim.Adam(Final_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

no_of_epoch=10
i=0
for epoch in range(no_of_epoch):
    loss_epoch = 0
    for batch_idx, (images, formulas) in enumerate(train_loader):
        i=i+1
        result=Final_model(images,formulas)
       
        print("result",result.shape)
        print("formula",formulas.shape)
        predicted_flat = result.reshape(result.shape[0]*result.shape[1],result.shape[2])
        actual_flat = formulas.reshape(formulas.shape[0]*formulas.shape[1])
        
        print("predicted_flat",predicted_flat.shape)
        print("actual_flat",actual_flat.shape)
    
        

        # Calculate cross-entropy loss
        #loss = nn.Cross_entropy(predicted_flat, actual_flat)
        loss = criterion(predicted_flat, actual_flat)
       # loss = F.cross_entropy(predicted_flat, actual_flat)
  
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()
        if(i%100):
            print("epoch ",epoch,i," ",loss," batch loss, ", loss_epoch/len(train_loader))
     

torch.save(Final_model.state_dict(), 'your_model_checkpoint.pth')# saving the model state

**loading the model and its state**

In [ ]:
Final_model = combined(Encoder(), Decoder(vocab_size, 512, 512))

# Load the saved model state dictionary
checkpoint_path = 'your_model_checkpoint.pth'  # Replace with the actual path
checkpoint = torch.load(checkpoint_path)

# Load the state dictionary into your model
Final_model.load_state_dict(checkpoint)


In [ ]:
row = train_dataset[0]
print(row[0].shape,row[1].shape)
test_image=row[0]
test_formula=row[1]

test_image=test_image.repeat(batch_size, 1, 1,1)
##result=Final_model
print(test_image.shape)

In [ ]:
predicted_formula=Final_model.predict(test_image)

In [ ]:
print(test_formula)
print(" ")
print(predicted_formuala)